## 初始化

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

print(sys.path)

['/mnt/workspace/code/ModelScopeGPT/agent/demo', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/tinycudann-1.6-py3.8-linux-x86_64.egg', '../']


## Load cfg and initialize

In [3]:
from dotenv import load_dotenv

load_dotenv('../config/.env', override=True)

import os
from modelscope.utils.config import Config

model_cfg_file = os.getenv('LLM_CONFIG_FILE')
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE') 

print(model_cfg_file)
print(tool_cfg_file)

model_cfg = Config.from_file(model_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

2023-08-03 14:38:23,599 - modelscope - INFO - PyTorch version 1.12.1+cu113 Found.
2023-08-03 14:38:23,604 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-08-03 14:38:23,634 - modelscope - INFO - Loading done! Current index file version is 1.7.0, with md5 bbb5a1af05c4dbbcda7f138f8a12f0bf and a total number of 861 components indexed


../config/cfg_model.json
../config/cfg_tool.json


In [4]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.llm.ms_gpt import ModelScopeGPT

llm = ModelScopeGPT(model_cfg)
agent = AgentExecutor(llm, tool_cfg)
agent.set_available_tools(['modelscope_image-generation'])

## 向量库配置

In [19]:
from langchain.embeddings import ModelScopeEmbeddings, DashScopeEmbeddings

from langchain.vectorstores import FAISS
import faiss


In [20]:
# embedding
# embedding_name = 'GanymedeNil/text2vec-large-chinese'
# embedding_device = 'cpu'
# model_id = 'damo/nlp_corom_sentence-embedding_chinese-base'
# embeddings = ModelScopeEmbeddings(model_id=model_id)
embeddings = DashScopeEmbeddings(model="text-embedding-v1")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 # embedding_device = 'cpu'                                                                   │
│   4 # model_id = 'damo/nlp_corom_sentence-embedding_chinese-base'                                │
│   5 # embeddings = ModelScopeEmbeddings(model_id=model_id)                                       │
│ ❱ 6 embeddings = DashScopeEmbeddings(model="text-embedding-v1")                                  │
│   7                                                                                              │
│                                                                                                  │
│ in pydantic.main.BaseModel.__init__:341                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for DashScopeEmbeddings
__root__
  Did not find dashscope_api_key, please add an environment variable `DASHSCOPE_API_KEY` which contains it, or pass
`dashscope_api_key` as a named parameter. (type=value_error)

In [9]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import re
from typing import List
from modelscope.pipelines import pipeline


# ref: https://github.com/chatchat-space/langchain-ChatGLM/blob/master/textsplitter/ali_text_splitter.py
class AliTextSplitter(CharacterTextSplitter):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def split_text(self, text: str) -> List[str]:

        p = pipeline(
            task="document-segmentation",
            model='damo/nlp_bert_document-segmentation_chinese-base',
            device="cpu")
        result = p(documents=text)
        sent_list = [i for i in result["text"].split("\n\t") if i]
        return sent_list


In [10]:
# 导入文件
filepaths = ['tmp/ms.txt', 'tmp/china.txt', 'tmp/xiyouji.txt']

def load_file(filepaths, sentence_size=100):
    textsplitter = AliTextSplitter()
    docs = []
    for filepath in filepaths:
        if not filepath.lower().endswith(".txt"):
            continue
        loader = TextLoader(filepath, autodetect_encoding=True)
        docs+=(loader.load_and_split(textsplitter))
    
    return docs
docs = load_file(filepaths)


2023-08-03 14:40:08,538 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
2023-08-03 14:40:08,713 - modelscope - INFO - initiate model from /mnt/workspace/.cache/modelscope/damo/nlp_bert_document-segmentation_chinese-base
2023-08-03 14:40:08,713 - modelscope - INFO - initiate model from location /mnt/workspace/.cache/modelscope/damo/nlp_bert_document-segmentation_chinese-base.
2023-08-03 14:40:08,716 - modelscope - INFO - initialize model from /mnt/workspace/.cache/modelscope/damo/nlp_bert_document-segmentation_chinese-base
2023-08-03 14:40:10,480 - modelscope - WARNING - No preprocessor field found in cfg.
2023-08-03 14:40:10,481 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-08-03 14:40:10,482 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/damo/nlp_bert_document-segmentation_ch

In [11]:

# 建立索引
vector_store = FAISS.from_documents(docs, embeddings)

In [12]:
top_k = 3
def search_query_wrapper(query):
    
    search_docs = vector_store.similarity_search(query, k=top_k)
    
    search_res = '\n'.join([f'[{idx+1}] {s.page_content}' for idx, s in enumerate(search_docs)])
    
    final_query = f'<|startofsearch|>\nsearch results: \n{search_res.strip()}\n<|endofsearch|>\n{query}'

    return final_query

In [13]:
query_without_search = '魔搭安装'
# query = '生成一个xxx的图片，用最强大的基模型'

query_with_search = search_query_wrapper(query_without_search)

print(query_with_search)

<|startofsearch|>
search results: 
[1] 7版本及以上。我们推荐您使用Anaconda进行安装。
[2] 安装深度学习框架。
[3] 安装ModelScope Library。我们提供两种安装方式，您可选择适合的方式进行安装。
<|endofsearch|>
魔搭安装


In [14]:
agent.reset()
agent.run(query_without_search)

|prompt1: <|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。
 当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}



<|user|>:魔搭安装

<|assistant|>:


**************************************************round 1**************************************************


<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8" />
    <meta
      name="viewport"
      content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no"
    />
    <meta property="og:title" content="ModelScope 魔搭社区" />
    <meta property="og:url" content="//modelscope.cn/" />
    <meta property="og:image" content="//img.alicdn.com/imgextra/i4/O1CN01fvt4it25rEZU4Gjso_!!6000000007579-2-tps-128-128.png" />
    <meta property="og:description" content="ModelScope——汇聚各领域先进的机器学习模型，提供模型探索体验、推理、训练、部署和应用的一站式服务。在这里，共建模型开源社区，发现、学习、定制和分享心仪的模型。" />

     <link rel="icon" type="image/x-icon" href="//g.alicdn.com/sail-web/maas/0.10.33/favicon/128.ico">
    <link
      rel="stylesheet"
      href="//g.alicdn.com/sail-web/maas/0.10.33/umi.css"
    />
    <script>
      window.routerBase = "/";
      window.publicPath = "//g.alicdn.com\/sail-web\/maas\/0.10.33/";
      window.__detail_data__ =  null ;
      window.MODELSCOPE_SITE_MODE = "COMMON";
      window.

[]

In [15]:
agent.reset()
agent.run(query_with_search)

|prompt1: <|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。
 当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}



<|user|>:
search results: 
[1] 7版本及以上。我们推荐您使用Anaconda进行安装。
[2] 安装深度学习框架。
[3] 安装ModelScope Library。我们提供两种安装方式，您可选择适合的方式进行安装。


魔搭安装

<|assistant|>:


**************************************************round 1**************************************************

根据搜索结果，我们建议您使用Anaconda进行安装深度学习框架，然后再安装ModelScope Library。您可以选择魔搭社区提供的两种安装方式之一进行安装。如果您想了解更多关于安装的细节，请参考搜索结果中的第一个链接。

[]

In [16]:
query_without_search = '画一张孙悟空的图'
query_with_search = search_query_wrapper(query_without_search)

print(query_with_search)

<|startofsearch|>
search results: 
[1] 猪八戒，法号猪悟能。他原为天界率领八万兵马的天蓬元帅，后来因酒醉后调戏嫦娥而被贬入凡间，转世成了猪脸人身的怪物，名为猪刚鬛。唐三藏与孙悟空来到高老庄，高老庄长老告诉他们，三年前招的女婿是妖怪，求唐僧孙悟空解救，于是孙悟空便变化成高翠兰，打倒猪刚鬛。当猪刚鬛得知孙悟空是唐三藏的弟子后，便主动投入三藏门下，并告知三藏自己也被观音菩萨告知必须以协助三藏取经来赎罪。猪八戒懂得三十六般变化，手持九齿钉耙作为武器。在旅途中，他好色、懒惰、窝囊的种种表现不时拖累师徒，更屡屡陷唐三藏于生死关头，但他偶尔也有优异的表现。在取得佛经后，猪八戒被封净坛使者菩萨，负责代替十方诸佛下凡享用香火供品。
[2] 	孙悟空孕育自一颗位于东胜神州花果山上的灵石，该石吸纳日月精华，内育仙胞，一日迸裂，产一石卵，见风化为石猴。它率领众猴在花果山称王，被敬拜为“美猴王”。它因见到其它猴子老死而兴起求得长生不死之法的念头，便前往斜月三星洞拜菩提祖师为师。菩提祖师赐它“孙悟空”之名，授他七十二般变化之法和斤斗云。
[3] 沙悟净如同猪八戒一样，沙悟净原本也是天界的一名将军。他本是卷帘大将，但因为不小心打破玻璃盏而让他受到玉皇大帝的惩罚，被贬入凡间后、他成了吃人妖怪。他住在流沙河中，袭击渡河船只。沙悟净的武器是降妖宝杖，他在吃人之后会将他们的头骨收集起来，无聊的时候把玩它们。后来观音菩萨告诉他、将会有位从东土而来的唐僧路经此地，观音要他拜唐僧为师。当唐僧和他的弟子来到流沙河时，沙悟净和孙悟空、猪八戒大打出手，当他得知他们是唐僧的弟子后，便拜唐僧为师共赴天竺。在旅途中，沙悟净沉默寡言、安份守己，尽管他在战斗上的表现不如两位师兄，但他是三名徒弟中对唐僧最忠诚有礼的。在取得佛经后，沙悟净被封为八宝金身罗汉菩萨。
<|endofsearch|>
画一张孙悟空的图


In [17]:
agent.reset()
agent.run(query_without_search, remote=True)

|prompt1: <|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。
 当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}



<|user|>:画一张孙悟空的图

<|assistant|>:


**************************************************round 1**************************************************

<IPython.core.display.JSON object>

调用ModelScope的图片生成模型生成中...


Traceback (most recent call last):
  File "/mnt/workspace/code/ModelScopeGPT/agent/demo/../modelscope_agent/agent.py", line 113, in run
    exec_result = tool(**action_args, remote=remote)
  File "/mnt/workspace/code/ModelScopeGPT/agent/demo/../modelscope_agent/tools/tool.py", line 57, in __call__
    return self._remote_call(*args, **kwargs)
  File "/mnt/workspace/code/ModelScopeGPT/agent/demo/../modelscope_agent/tools/text_to_image_tool.py", line 32, in _remote_call
    response = ImageSynthesis.call(model=ImageSynthesis.Models.wanx_v1,
  File "/opt/conda/lib/python3.8/site-packages/dashscope/aigc/image_synthesis.py", line 49, in call
    return super().call(model, prompt, negative_prompt, images, api_key,
  File "/opt/conda/lib/python3.8/site-packages/dashscope/client/base_api.py", line 172, in call
    response = cls.wait(task_response)
  File "/opt/conda/lib/python3.8/site-packages/dashscope/aigc/image_synthesis.py", line 129, in wait
    response = super().wait(task, api_key)
  F

[{'error': "Action call error: modelscope_image-generation: {'text': '一个英勇的孙悟空，他手持金箍棒，身披红色斗篷，英姿飒爽，4K分辨率，色彩鲜艳，立体派风格，让你感受到孙悟空的英雄气概。'}."}]

In [18]:
agent.reset()
agent.run(query_with_search, remote=True)

|prompt1: <|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。
 当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}



<|user|>:
search results: 
[1] 猪八戒，法号猪悟能。他原为天界率领八万兵马的天蓬元帅，后来因酒醉后调戏嫦娥而被贬入凡间，转世成了猪脸人身的怪物，名为猪刚鬛。唐三藏与孙悟空来到高老庄，高老庄长老告诉他们，三年前招的女婿是妖怪，求唐僧孙悟空解救，于是孙悟空便变化成高翠兰，打倒猪刚鬛。当猪刚鬛得知孙悟空是唐三藏的弟子后，便主动投入三藏门下，并告知三藏自己也被观音菩萨告知必须以协助三藏取经来赎罪。猪八戒懂得三十六般变化，手持九齿钉耙作为武器。在旅途中，他好色、懒惰、窝囊的种种表现不时拖累师徒，更屡屡陷唐三藏于生死关头，但他偶尔也有优异的表现。在取得佛经后，猪八戒被封净坛使者菩萨，负责代替十方诸佛下凡享用香火供品。
[2] 	孙悟空孕育自一颗位于东胜神州花果山上的灵石，该石吸纳日月精华，内育仙胞，一日迸裂，产一石卵，见风化为石猴。它率领众猴在花果山称王，被敬拜为“美猴王”。它因见到其它猴子老死而兴起求得长生不死之法的念头，便前往斜月三星洞拜菩提祖师为师。菩提祖师赐它“孙悟空”之名，授他七十二般变化之法和斤斗云。
[3] 沙悟净如同猪八戒一样，沙悟净原本也是天界的一名将军。他本是卷帘大将，但因

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

调用AI画图模型生成中...


|prompt2: <|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。
 当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}



<|user|>:
search results: 
[1] 猪八戒，法号猪悟能。他原为天界率领八万兵马的天蓬元帅，后来因酒醉后调戏嫦娥而被贬入凡间，转世成了猪脸人身的怪物，名为猪刚鬛。唐三藏与孙悟空来到高老庄，高老庄长老告诉他们，三年前招的女婿是妖怪，求唐僧孙悟空解救，于是孙悟空便变化成高翠兰，打倒猪刚鬛。当猪刚鬛得知孙悟空是唐三藏的弟子后，便主动投入三藏门下，并告知三藏自己也被观音菩萨告知必须以协助三藏取经来赎罪。猪八戒懂得三十六般变化，手持九齿钉耙作为武器。在旅途中，他好色、懒惰、窝囊的种种表现不时拖累师徒，更屡屡陷唐三藏于生死关头，但他偶尔也有优异的表现。在取得佛经后，猪八戒被封净坛使者菩萨，负责代替十方诸佛下凡享用香火供品。
[2] 	孙悟空孕育自一颗位于东胜神州花果山上的灵石，该石吸纳日月精华，内育仙胞，一日迸裂，产一石卵，见风化为石猴。它率领众猴在花果山称王，被敬拜为“美猴王”。它因见到其它猴子老死而兴起求得长生不死之法的念头，便前往斜月三星洞拜菩提祖师为师。菩提祖师赐它“孙悟空”之名，授他七十二般变化之法和斤斗云。
[3] 沙悟净如同猪八戒一样，沙悟净原本也是天界的一名将军。他本是卷帘大将，但因

**************************************************round 2**************************************************

![IMAGEGEN](https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1e5e2015/20230803/1450/ab436d8a-728d-4a06-aebd-70fa332fa255-1.png?Expires=1691131822&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=I9M5EqPPcwtv6W%2FLB%2Fm6%2FEu7R8w%3D)

[{'result': ![IMAGEGEN](https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1e5e2015/20230803/1450/ab436d8a-728d-4a06-aebd-70fa332fa255-1.png?Expires=1691131822&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=I9M5EqPPcwtv6W%2FLB%2Fm6%2FEu7R8w%3D)}]